In [ ]:
import pandas as pd
import pickle
from janome.tokenizer import Tokenizer
from tqdm.auto import tqdm

# ===================================================================
# ユーザー定義関数（ここから）
# ===================================================================
wakati = Tokenizer()

def tokenize_ja(sentences_list):
    """日本語文のトークン化"""
    wakati_list = []
    print("トークン化処理を開始します...")
    for sentence in tqdm(sentences_list):
        wakati_list.append([item.surface for item in wakati.tokenize(sentence)])
    return wakati_list

def create_word_id_dict(sentences):
    """単語からIDへの辞書を生成"""
    word_to_id, id_to_word = {}, {}
    # 0はパディング/未知語用に予約
    word_to_id['<PAD>/<UNK>'] = 0
    id_to_word[0] = '<PAD>/<UNK>'
    
    for sentence in sentences:
        for word in sentence:
            if word not in word_to_id:
                tmp_id = len(word_to_id)
                word_to_id[word] = tmp_id
                id_to_word[tmp_id] = word
    return word_to_id, id_to_word

def convert_sentences_to_ids(sentences, word_to_id):
    """文章をID列に変換"""
    sentence_id = []
    for sentence in sentences:
        sentence_ids = [word_to_id.get(word, 0) for word in sentence] # .getで高速化
        sentence_id.append(sentence_ids)
    return sentence_id

def padding_sentence(sentences):
    """文章のパディング処理"""
    max_len = max(len(s) for s in sentences) if sentences else 0
    
    padded_sentences = []
    for sentence in sentences:
        padding = [0] * (max_len - len(sentence))
        padded_sentences.append(padding + sentence)
    return padded_sentences

# ===================================================================
# ユーザー定義関数（ここまで）
# ===================================================================

# --- メイン処理 ---
print("1. 生データの読み込み...")
df = pd.read_csv('livedoor_news_corpus.csv') # 事前に作成したCSV

# --- ラベルのID化 ---
print("2. ラベルのID化...")
label_to_id = {label: i for i, label in enumerate(df['label'].unique())}
id_to_label = {i: label for i, label in enumerate(df['label'].unique())}
df['label_id'] = df['label'].map(label_to_id)

# --- テキストの前処理 ---
# 3. テキストのトークン化（分かち書き）
ja_sentences = tokenize_ja(df['text'].tolist())

# 4. 単語辞書の作成
print("4. 単語辞書の作成...")
word_to_id, id_to_word = create_word_id_dict(ja_sentences)

# 5. 文章をID列に変換
print("5. 文章をID列に変換...")
sentence_ids = convert_sentences_to_ids(ja_sentences, word_to_id)

# 6. パディング処理
print("6. パディング処理...")
padded_ids = padding_sentence(sentence_ids)

# --- データの保存 ---
print("7. 処理済みデータの保存...")
processed_data = {
    'padded_ids': padded_ids,
    'labels': df['label_id'].tolist(),
    'word_to_id': word_to_id,
    'id_to_word': id_to_word,
    'label_to_id': label_to_id,
    'id_to_label': id_to_label,
}

with open('processed_data.pkl', 'wb') as f:
    pickle.dump(processed_data, f)

print("\n🎉 前処理とデータの保存が完了しました。")
print(f"保存ファイル: processed_data.pkl")
print(f"語彙数: {len(word_to_id)}")

1. 生データの読み込み...
2. ラベルのID化...
トークン化処理を開始します...


  0%|          | 0/7376 [00:00<?, ?it/s]